In [5]:
import requests

base_url = 'https://book-review-api2.onrender.com/uppercase'
params= {'text': 'HelloStill Active?th'}
response = requests.get(base_url, params=params)

# Print the JSON content from the response
if response.status_code == 200:
    print(response.json())  # Call the method with parentheses
else:
    print(f"Request failed with status code {response.status_code}")

{'text': 'HELLOSTILL ACTIVE?TH'}


In [21]:
from flask import Flask, jsonify, request
from flask_restful import Api, Resource
from flasgger import Swagger
import book_review

app = Flask(__name__)
api = Api(app)
swagger = Swagger(app)

br= book_review.Book_review()




In [23]:


class UppercaseText(Resource):
    def get(self):
        """
        This method responds to the GET request for this endpoint and returns the data in uppercase.
        ---
        tags:
        - Text Processing
        parameters:
            - name: text
              in: query
              type: string
              required: true
              description: The text to be converted to uppercase
        responses:
            200:
                description: A successful GET request
                content:
                    application/json:
                      schema:
                        type: object
                        properties:
                            text:
                                type: string
                                description: The text in uppercase
        """
        text = request.args.get('text')

        return jsonify({"text": text.upper()})
    
class AllReviews(Resource):
    def get(self):
        """
        This method responds to the GET request for the list of book reviews.
        ---
        tags:
        - Book Reviews
        parameters:
            - name: sort
              in: query
              type: string
              enum: [ASC, DESC]
              required: false
              description: Sort order for reviews (ASC or DESC)
            - name: max_records
              in: query
              type: integer
              required: false
              description: Maximum number of records to return
        responses:
            200:
                description: A successful GET request
                content:
                    application/json:
                      schema:
                        type: array
                        items:
                            type: object
                            properties:
                                title:
                                    type: string
                                    description: The title of the book
                                review:
                                    type: string
                                    description: The review of the book
                                rating:
                                    type: integer
                                    description: The rating of the book
        """
        sort = request.args.get('sort', default=None)
        max_records = int(request.args.get('max_records', default=10))

        # Check for valid sort parameter
        if sort and sort.lower() not in ["asc", "desc"]:
            return {"error": "Invalid sort value"}, 400

        # Sort the book reviews if sort parameter is provided
        if sort =="ASC":
            book_review = br.get_book_rating(sort=sort, max_records=max_records)
        elif sort =="DESC":
            book_review = br.get_book_rating(sort=sort, max_records=max_records)
        else: 
            book_review = br.get_book_rating(max_records=max_records)


        return book_reviews, 200




    
class AddRecord(Resource):
    def post(self):
        """
        This method responds to the POST request for adding a new record to the DB table.
        ---
        tags:
        - Records
        parameters:
            - in: body
              name: body
              required: true
              schema:
                id: BookReview
                required:
                  - Book
                  - Rating
                properties:
                  Book:
                    type: string
                    description: the name of the book
                  Rating:
                    type: integer
                    description: the rating of the book (1-10)
        responses:
            200:
                description: A successful POST request
            400: 
                description: Bad request, missing 'Book' or 'Rating' in the request body
        """

        data = request.json
        print(data)

        # Check if 'Book' and 'Rating' are present in the request body
        if 'Book' not in data or 'Rating' not in data:
            return {"message": "Bad request, missing 'Book' or 'Rating' in the request body"}, 400
        # Call the add_record function to add the record to the DB table
        success = book_review.add_record(data)

        if success:
            return {"message": "Record added successfully"}, 200
        else:
            return {"message": "Failed to add record"}, 500


api.add_resource(AllReviews, "/all_reviews")
api.add_resource(AddRecord, "/add-record")
api.add_resource(UppercaseText, "/uppercase")


In [18]:
br.get_book_rating(sort="ASC", max_records=4)

[{'id': 'recNwG1gQWXNpA6R4',
  'createdTime': '2024-08-04T08:39:36.000Z',
  'fields': {'Book': 'Investing 101',
   'Rating': 5,
   'Notes': 'Just something you can find on the internet'}},
 {'id': 'recUiuiMH5RghCANY',
  'createdTime': '2024-08-04T10:21:36.000Z',
  'fields': {'Book': 'Investing 101',
   'Rating': 5,
   'Notes': 'Just something you can find on the internet'}},
 {'id': 'recrIYBfRLqaSwjHP',
  'createdTime': '2024-08-04T10:34:57.000Z',
  'fields': {'Book': 'Investing 101',
   'Rating': 5,
   'Notes': 'Just something you can find on the internet'}},
 {'id': 'recx9HYuNEtjEJffw',
  'createdTime': '2024-08-04T08:04:20.000Z',
  'fields': {'Book': 'The World Worst Assistance by Sona Movsessian',
   'Rating': 7.4}}]